In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import newton

# Set up the environment for displaying output
np.set_printoptions(threshold=np.inf, linewidth=200, suppress=True)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.random.seed(9)

In [2]:
# Parameters
r0 = 0.05
r_mean = 0.05
sd = 0.1
kappa = 0.82
F = 1000  # Face value of the bond

In [3]:
# Vasicek model for short-rate paths
def generate_vasicek_paths(initial_rate, maturity, num_paths, num_steps):
    dt = maturity / num_steps
    rates = np.zeros((num_paths, num_steps + 1))
    rates[:, 0] = initial_rate
    for step in range(1, num_steps + 1):
        rates[:, step] = rates[:, step - 1] + kappa * (r_mean - rates[:, step - 1]) * dt + \
                         sd * np.sqrt(dt) * np.random.normal(0, 1, num_paths)
    return rates

# Zero-Coupon Bond price under the Vasicek model
def zero_coupon_bond_vasicek(initial_rate, t, maturity, face_value):
    num_paths = 1000
    num_steps = int(365 * (maturity - t))
    dt = (maturity - t) / num_steps
    short_rates = generate_vasicek_paths(initial_rate, maturity - t, num_paths, num_steps)
    bond_prices = np.mean(face_value * np.exp(-np.sum(short_rates, axis=1) * dt))
    return bond_prices

print("Price of Zero Coupon Bond under Vasicek: $", round(zero_coupon_bond_vasicek(r0, 0, 0.5, F), 5))

Price of Zero Coupon Bond under Vasicek: $ 976.01367


In [4]:
def coupon_bond_vasicek(initial_rate, t, coupons, payment_dates):
    zero_coupon_prices = [zero_coupon_bond_vasicek(initial_rate, t, maturity, coupon) 
                          for coupon, maturity in zip(coupons, payment_dates)]
    return np.sum(zero_coupon_prices)

coupons = [30] * 7 + [1030]
payment_dates = np.arange(0.5, 4.5, 0.5)
print("Price of Coupon Paying Bond under Vasicek: $", coupon_bond_vasicek(r0, 0, coupons, payment_dates))

Price of Coupon Paying Bond under Vasicek: $ 1052.3168050458744


In [6]:
#  European call option on zero coupon bond using explicit formula
def zero_coupon_bond_explicit(short_rate, t, maturity, face_value):
    B = (1 / kappa) * (1 - np.exp(-kappa * (maturity - t)))
    A = np.exp(((r_mean - (sd ** 2) / (2 * kappa ** 2)) * (B - (maturity - t)) - (sd ** 2) / (4 * kappa) * B ** 2))
    return face_value * A * np.exp(-B * short_rate)

def european_call_option_explicit(t, maturity, strike_price, face_value):
    num_paths = 15000
    num_steps = int(365 * (maturity - t))
    dt = (maturity - t) / num_steps
    short_rates = generate_vasicek_paths(r0, t, num_paths, num_steps)
    bond_prices = zero_coupon_bond_explicit(short_rates[:, -1], t, maturity, face_value)
    payoffs = np.maximum(bond_prices - strike_price, 0)
    option_price = np.mean(payoffs * np.exp(-np.sum(short_rates, axis=1) * dt))
    return option_price

print("Price of European Call Option on Zero Coupon Bond (Explicit Formula): $", 
      round(european_call_option_explicit(0.25, 0.5, 980, F), 5))

Price of European Call Option on Zero Coupon Bond (Explicit Formula): $ 8.87486


In [ ]:
# European call option on coupon bond using Monte Carlo simulation
def european_call_option_coupon_monte_carlo(t, strike_price, face_value, coupons, payment_dates):
    num_paths = 1000
    num_steps = int(365 * t)
    dt = t / num_steps
    short_rates = generate_vasicek_paths(r0, t, num_paths, num_steps)
    bond_prices = np.array([coupon_bond_vasicek(rate, t, coupons, payment_dates) for rate in short_rates[:, -1]])
    payoffs = np.maximum(bond_prices - strike_price, 0)
    option_price = np.mean(payoffs * np.exp(-np.sum(short_rates, axis=1) * dt))
    return option_price

print("Price of European Call Option on Coupon Paying Bond (Monte Carlo Simulation): $", 
      round(european_call_option_coupon_monte_carlo(0.25, 980, F, coupons, payment_dates), 5))

In [ ]:
# Using the Black-Scholes formula for the European option on coupon bond
def black_scholes_coupon_option(t, maturity, strike_price, coupons, payment_dates):
    def f_opt(rate, maturity, payment_dates, coupons, strike_price):
        PIs = np.array([PI(rate, maturity, payment_date) for payment_date in payment_dates])
        return np.sum(coupons * PIs) - strike_price

    def PI(rate, maturity, payment_date):
        B = (1 / kappa) * (1 - np.exp(-kappa * (payment_date - maturity)))
        A = np.exp((r_mean - (sd ** 2) / (2 * kappa ** 2)) * (B - (payment_date - maturity)) - (sd ** 2) / (4 * kappa) * B ** 2)
        return A * np.exp(-B * rate)

    r_star = newton(f_opt, 0, args=(maturity, payment_dates, coupons, strike_price))
    PIs = np.array([PI(r_star, maturity, payment_date) for payment_date in payment_dates])

    P_t_T = np.array([PI(r0, 0, payment_date) for payment_date in payment_dates])
    sd_p = sd * (1 - np.exp(-kappa * (payment_dates - maturity)) / kappa) * np.sqrt((1 - np.exp(-2 * kappa * maturity)) / (2 * kappa))

    d1 = np.log(P_t_T[0] / (strike_price * P_t_T[1])) / sd_p + sd_p / 2
    d2 = d1 - sd_p

    option_price = np.sum(coupons * (P_t_T[0] * norm.cdf(d1) - strike_price * P_t_T[1] * norm.cdf(d2)))
    return option_price

print("Price of European Call Option on Coupon Paying Bond (Black-Scholes Formula): $", 
      round(black_scholes_coupon_option(0.25, 0.5, 980, coupons, payment_dates), 5))

In [ ]:
# (Cox-Ingersoll-Ross Model for interest rates)

# European Call Option on Zero Coupon Bond under CIR model using Monte Carlo simulations
def generate_cir_paths(initial_rate, sd, kappa, r_mean, maturity, num_paths, num_steps):
    dt = maturity / num_steps
    rates = np.zeros((num_paths, num_steps + 1))
    rates[:, 0] = initial_rate
    for step in range(1, num_steps + 1):
        rates[:, step] = rates[:, step - 1] + kappa * (r_mean - rates[:, step - 1]) * dt + \
                         sd * np.sqrt(rates[:, step - 1]) * np.sqrt(dt) * np.random.normal(0, 1, num_paths)
    return rates

def zero_coupon_bond_cir(initial_rate, sd, kappa, r_mean, maturity, face_value):
    num_paths = 1000
    num_steps = int(365 * maturity)
    dt = maturity / num_steps
    short_rates = generate_cir_paths(initial_rate, sd, kappa, r_mean, maturity, num_paths, num_steps)
    bond_prices = np.mean(face_value * np.exp(-np.sum(short_rates, axis=1) * dt))
    return bond_prices

def european_call_option_cir(initial_rate, sd, kappa, r_mean, maturity, strike_price, face_value):
    num_paths = 1200
    num_steps = int(365 * maturity)
    dt = maturity / num_steps
    short_rates = generate_cir_paths(initial_rate, sd, kappa, r_mean, maturity, num_paths, num_steps)
    bond_prices = np.array([zero_coupon_bond_cir(rate, sd, kappa, r_mean, maturity, face_value) for rate in short_rates[:, -1]])
    payoffs = np.maximum(bond_prices - strike_price, 0)
    option_price = np.mean(payoffs * np.exp(-np.sum(short_rates, axis=1) * dt))
    return option_price

print("Price of European Call Option on Zero Coupon Bond under CIR (Monte Carlo Simulation): $", 
      round(european_call_option_cir(r0, sd, kappa, r_mean, 0.5, 980, F), 5))